In [1]:
from imutils import paths
import face_recognition
import argparse
import pickle
import cv2
import os
from imutils.video import VideoStream
import face_recognition
import imutils
import pickle
import time
import csv
import pandas as pd

In [2]:
# heading_frame = pd.DataFrame(columns={'a','b','c'})
# heading_frame.to_csv(os.path.join('Databse_and_records','main_database_record.csv'),index=False)
# print(heading_frame)
# os.path.isfile(os.path.join('Database_and_records','main_database_record.csv'
# ?pd.DataFrame
# heading_frame = pd.DataFrame(columns=['Id_number','Name','Contact_no','Permission'])
# print(heading_frame)
# encodedImage = base64.b64encode(open(os.path.join('dataset','Mansi123','Mansi.1.jpg'), 'rb').read()
# Data['HYPERLINK'] = "<a href=\"" + Data['Image File'] + "\">"+Data['Param']+"</a>"

In [3]:
data_path=os.path.join('dataset')
encodings_path=os.path.join("encodings.pickle")
model='hog'
output='video'
display=1

In [4]:
def image_paths_finder(data_path):
    print("[INFO] quantifying faces...")
    imagePaths = list(paths.list_images(data_path))
    return imagePaths


In [5]:
def image_encoder(data_path,encodings_path,model,imagePaths):
    knownEncodings = []
    knownNames = []

            # loop over the image paths
    for (i, imagePath) in enumerate(imagePaths):
                    # extract the person name from the image path
        print("[INFO] processing image {}/{}".format(i + 1,len(imagePaths)))
        name = imagePath.split(os.path.sep)[-2]
                    

                    # load the input image and convert it from RGB (OpenCV ordering)
                    # to dlib ordering (RGB)

        image = cv2.imread(imagePath)
        rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    # detect the (x, y)-coordinates of the bounding boxes
                    # corresponding to each face in the input image
        boxes = face_recognition.face_locations(rgb,
        model=model)

                    # compute the facial embedding for the face
        encodings = face_recognition.face_encodings(rgb, boxes)

                    # loop over the encodings
        for encoding in encodings:
                            # add each encoding + name to our set of known names and
                           # encodings
            knownEncodings.append(encoding)
            knownNames.append(name)

            # dump the facial encodings + names to disk
    print("[INFO] serializing encodings...")
    data = {"encodings": knownEncodings, "names": knownNames}
    f = open(encodings_path, "wb")
    f.write(pickle.dumps(data))
    f.close()
    image_detector(model,encodings_path,output,display)
    return    

In [6]:
def image_detector(model,encodings,output,display):
    count=0
    print("[INFO] loading encodings...")
    data = pickle.loads(open(encodings, "rb").read())
    print("[INFO] starting video stream...")
    vs = VideoStream(src=0).start()
    writer = None
    time.sleep(2.0)   
    # loop over frames from the video file stream
    while True:
        # grab the frame from the threaded video stream
        frame = vs.read()

        # convert the input frame from BGR to RGB then resize it to have
        # a width of 750px (to speedup processing)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        rgb = imutils.resize(frame, width=750)
        r = frame.shape[1] / float(rgb.shape[1])

        # detect the (x, y)-coordinates of the bounding boxes
        # corresponding to each face in the input frame, then compute
        # the facial embeddings for each face
        boxes = face_recognition.face_locations(rgb,model=model)
        encodings = face_recognition.face_encodings(rgb, boxes)
        names = []
        # loop over the facial embeddings
        for encoding in encodings:
            # attempt to match each face in the input image to our known
            # encodings
            matches = face_recognition.compare_faces(data["encodings"],encoding)
            name = "Unknown"

            # check to see if we have found a match
            if True in matches:
                # find the indexes of all matched faces then initialize a
                # dictionary to count the total number of times each face
                # was matched
                matchedIdxs = [i for (i, b) in enumerate(matches) if b]
                counts = {}

                # loop over the matched indexes and maintain a count for
                # each recognized face face
                for i in matchedIdxs:
                    name = data["names"][i]
                    counts[name] = counts.get(name, 0) + 1

                # determine the recognized face with the largest number
                # of votes (note: in the event of an unlikely tie Python
                # will select first entry in the dictionary)
                name = max(counts, key=counts.get)

            # update the list of names
            names.append(name)
        
        # loop over the recognized faces
        for ((top, right, bottom, left), name) in zip(boxes, names):
            # rescale the face coordinates
            top = int(top * r)
            right = int(right * r)
            bottom = int(bottom * r)
            left = int(left * r)

            # draw the predicted face name on the image
            cv2.rectangle(frame, (left, top), (right, bottom),
                (0, 255, 0), 2)
            y = top - 15 if top - 15 > 15 else top + 15
            cv2.putText(frame, name, (left, y), cv2.FONT_HERSHEY_SIMPLEX,
                0.75, (0, 255, 0), 2)
            # Activity on detection of a unknown entry is here
            if name == 'Unknown':
                count=count+1
                if count > 10:
                    vs.stop()
                    vs.stream.release()
                    cv2.destroyAllWindows()
                    security_process(frame) 
            else:
                Entry_log_maker(name)
        # if the video writer is None *AND* we are supposed to write
        # the output video to disk initialize the writer
        if writer is None and output is not None:
            fourcc = cv2.VideoWriter_fourcc(*"MJPG")
            writer = cv2.VideoWriter(output, fourcc, 20,
                (frame.shape[1], frame.shape[0]), True)

        # if the writer is not None, write the frame with recognized
        # faces t odisk
        if writer is not None:
            writer.write(frame)

        # check to see if we are supposed to display the output frame to
        # the screen
        if display > 0:
            cv2.imshow("Frame", frame)
            key = cv2.waitKey(1) & 0xFF

            # if the `q` key was pressed, break from the loop
            if key == ord("q"):
                break

    # do a bit of cleanup
    cv2.destroyAllWindows()
    vs.stop()

    # check to see if the video writer point needs to be released
    if writer is not None:
        writer.release()
    return    

In [7]:
def security_process(conflict_image):
    cv2.imshow('unknown',conflict_image)
    print('what is your name')
    name=input()
    correct = 'No'
    while correct != 'yes':
        print("Enter your contact number please")
        contact_no = input()
        total_no=len(list(contact_no))
        list_number=list(contact_no)
        alphabets = []
        if total_no == 10 :
            for word in range(total_no):
                if list_number[word].isalpha():
                    alphabets.append(list_number[word])
            if alphabets == []:
                print("Thanks for entering valid mobile no")
                correct = 'yes'
        else:
            print("Invalid number entered ")
            print("enter the number again")
    print('Allow the stranger to come in the premises(y,n):')
    permission=input()
    cv2.destroyAllWindows()
    
#######################    
    from firebase import firebase
    from datetime import datetime
    import pytz

    ist = pytz.timezone('Asia/Calcutta')

    print('Current Date Time in IST =', datetime.now(ist))
    print(datetime.date(datetime.now()))
    print(datetime.time(datetime.now()))

    firebase = firebase.FirebaseApplication("https://drishyam-2ae0f.firebaseio.com/",None)
    data={
        'Id':'1',
        'Name': 'Mansi',
        'Phone': 9999999999,
        'Arrival-Date':'2020-01-01',
        'Arrival-Time':11,
        'Exit-Time':12,
        'No-of-Arrivals':1
        }

    result = firebase.post('/drishyam-2aeOf/People',data)
    print(result)
##########################    
    if permission == 'n' or permission =='N':
        main_database_storge(name,conflict_image,contact_no,"not Allowed","None")
        print("sorry you are not allowed to enter")
    else:
        print("enter unique Id for the new user")
        Id = input()
        main_database_storage(name,conflict_image,contact_no,"Allowed",Id)
        model_database_maker(name,data_path,Id)
    return    

In [8]:
def model_database_maker(name,dataset_path,Identification_no):
    #making new folder in databse
    print('please stand on the cross')
    os.mkdir(os.path.join(dataset_path,name+str(Identification_no)))
    faceDetect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");
    cam = cv2.VideoCapture(0);
    time.sleep(2.0) 
    sampleNo = 0;
    while (sampleNo <= 50):
        ret,img = cam.read();
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY);
        faces = faceDetect.detectMultiScale(gray, 1.3, 5);
        for(x,y,w,h) in faces:
            sampleNo = sampleNo+1;
            cv2.imwrite(os.path.join(dataset_path,name+str(Identification_no),name+'.'+str(sampleNo)+'.jpg'),img[y:y+h,x:x+w])
            print(dataset_path+'\\'+name+str(Identification_no)+'\\'+str(name)+"."+str(sampleNo)+".jpg")
#             dataset_path+'\\'+name+str(Identification_no)+'\\'+str(name)+"."+str(sampleNo)+".jpg"
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
            cv2.waitKey(100)
        cv2.imshow("Face", img);
        cv2.waitKey(1)
    cam.release()
    cv2.destroyAllWindows()
    image_encoder(data_path,encodings_path,model,image_paths_finder(data_path))
    return

In [9]:
def main_database_storage(name,conflict_image,contact_no,allowance,Identification):
    cv2.imwrite(os.path.join('Database_and_records','Total_unique_faces',name+'.'+str(Identification)+'.jpg'),conflict_image)
    file_exist='No'
    while file_exist == 'No':
        if os.path.isfile(os.path.join('Database_and_records','main_database_record.csv')):
            file_exist = 'yes'
            with open(os.path.join('Database_and_records','main_database_record.csv'), 'a', newline='') as file:
                writer = csv.writer(file)
                HYPERLINK = "<a href=\"" +os.path.join('Database_and_records','main_database_record.csv')+ "\">"+"</a>"
                writer.writerow([Identification,name,contact_no,allowance,HYPERLINK])
                file.close()
        else:
            heading_frame = pd.DataFrame(columns=['Id_number','Name','Contact_no','Permission','Image_link'])
            heading_frame.to_csv(os.path.join('Database_and_records','main_database_record.csv'),index=False)
            file_exist = 'No'
    return

In [10]:
def Entry_log_maker(name):
    df_m = pd.read_csv(os.path.join('Database_and_records','main_database_record.csv'))
    df_l = pd.read_excel(os.path.join('Database_and_records','log_register.xlsx'))
    a = name
    b = a.rsplit('_', 1)[1]
    for i in range(df_m.shape[0]):
        if(b == str(df_m.iloc[i,df_m.columns.get_loc('id')])):

            today = date.today()
            now = datetime.now()
            d3 = today.strftime("%Y-%m-%d")
            time = now.strftime("%H:%M:%S")

            for j in range(df_l.shape[0]):
                if(b == str(df_l.iloc[j,df_l.columns.get_loc('id')]) and (d3 == str((df_l.iloc[j,df_l.columns.get_loc('date')]).date()) and str(df_l.iloc[j,df_l.columns.get_loc('permission')]) == 'allowed')):
                    total_time=(datetime.strptime(time,'%H:%M:%S') - datetime.strptime(str(df_l.iloc[j,df_l.columns.get_loc('time')]),'%H:%M:%S'))
                    if(total_time.total_seconds() <= 120.0):    
                        break

                    else:
                        print ('matched', i)
                        df_l = df_l.append({'id': df_m.iloc[i,df_m.columns.get_loc('id')], 'name': df_m.iloc[i,df_m.columns.get_loc('name')], 'number': df_m.iloc[i,df_m.columns.get_loc('number')], 'image': df_m.iloc[i,df_m.columns.get_loc('image')], 'permission': df_m.iloc[i,df_m.columns.get_loc('permission')], 'date': d3, 'time': time}, ignore_index=True)
                        df_l.to_excel(os.path.join('Database_and_records','log_register.xlsx'),index=False)
                          
    return   

                        

In [11]:
#image_encoder(data_path,encodings_path,model,image_paths_finder(data_path))
image_detector(model,encodings_path,output,display)

[INFO] loading encodings...
[INFO] starting video stream...


IndexError: list index out of range

In [ ]:
database_storage_unknown

# pagal ho gaya mai

# Hurray !!!!!!!    ab 100% detect hogi tu


In [ ]:
dakjdnakldnlakdjadjasdjasdjasdjlakdsjalksdjalkdjakdhakjdhakdb